# Bilinear terms

**Adapted from**: Section 3.1 of [F99] and Table 5.1 of [Las09]

[F99] Floudas, Christodoulos A. et al.
*Handbook of Test Problems in Local and Global Optimization.*
Nonconvex Optimization and Its Applications (NOIA, volume 33).

[Las09] Lasserre, J. B.
*Moments, positive polynomials and their applications*
World Scientific, **2009**.

## Introduction

Consider the polynomial optimization problem [F99, Section 3.1]

In [1]:
using DynamicPolynomials
@polyvar x[1:8]
p = sum(x[1:3])
using SumOfSquares
K = @set 0.0025 * (x[4] + x[6]) <= 1 &&
    0.0025 * (-x[4] + x[5] + x[7]) <= 1 &&
    0.01 * (-x[5] + x[8]) <= 1 &&
    100x[1] - x[1] * x[6] + 8333.33252x[4] <= 250000/3 &&
    x[2] * x[4] - x[2] * x[7] - 1250x[4] + 1250x[5] <= 0 &&
    x[3] * x[5] - x[3] * x[8] - 2500x[5] + 1250000 <= 0 &&
    100 <= x[1] && x[1] <= 10000 &&
    1000 <= x[2] && x[2] <= 10000 &&
    1000 <= x[3] && x[3] <= 10000 &&
    10 <= x[4] && x[4] <= 1000 &&
    10 <= x[5] && x[5] <= 1000 &&
    10 <= x[6] && x[6] <= 1000 &&
    10 <= x[7] && x[7] <= 1000 &&
    10 <= x[8] && x[8] <= 1000

Basic semialgebraic Set defined by no equality
22 inequalities
 1.0 - 0.0025*x[6] - 0.0025*x[4] ≥ 0
 1.0 - 0.0025*x[7] - 0.0025*x[5] + 0.0025*x[4] ≥ 0
 1.0 - 0.01*x[8] + 0.01*x[5] ≥ 0
 83333.33333333333 - 8333.33252*x[4] - 100.0*x[1] + x[1]*x[6] ≥ 0
 -1250.0*x[5] + 1250.0*x[4] + x[2]*x[7] - x[2]*x[4] ≥ 0
 -1.25e6 + 2500.0*x[5] + x[3]*x[8] - x[3]*x[5] ≥ 0
 -100.0 + x[1] ≥ 0
 10000.0 - x[1] ≥ 0
 -1000.0 + x[2] ≥ 0
 10000.0 - x[2] ≥ 0
 -1000.0 + x[3] ≥ 0
 10000.0 - x[3] ≥ 0
 -10.0 + x[4] ≥ 0
 1000.0 - x[4] ≥ 0
 -10.0 + x[5] ≥ 0
 1000.0 - x[5] ≥ 0
 -10.0 + x[6] ≥ 0
 1000.0 - x[6] ≥ 0
 -10.0 + x[7] ≥ 0
 1000.0 - x[7] ≥ 0
 -10.0 + x[8] ≥ 0
 1000.0 - x[8] ≥ 0


We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [2]:
import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following function searches for the largest lower bound and finds zero using the `d`th level of the hierarchy`.

In [3]:
function solve(d)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Max, α)
    @constraint(model, c, p >= α, domain = K, maxdegree = d)
    optimize!(model)
    println(solution_summary(model))
    return model
end

solve (generic function with 1 method)

The first level of the hierarchy gives a lower bound of `2100`

In [4]:
model2 = solve(2)

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 21
  constraints   = 29
  nnz(P)        = 0
  nnz(A)        = 64
  cones (total) = 2
    : Zero        = 1,  numel = 9
    : Nonnegative = 1,  numel = 20

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap  

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: VariableRef
`Vector{AffExpr}`-in-`SumOfSquares.SOSPolynomialSet{BasicSemialgebraicSet{Float64, DynamicPolynomials.Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}, Float64}, FullSpace}, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, SumOfSquares.Certificate.Putinar{SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, SumOfSquares.Certificate.NewtonFilter{SumOfSquares.Certificate.NewtonDegreeBounds{Tuple{}}}}, SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, SumOfSquares.Certificate.NewtonFilter{SumOfSquares.Certificate.NewtonDegreeBo

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*